## Загрузка библиотек   

In [16]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

## Загрузка данных для работы

In [6]:
# загрузка данных
dataset = pd.read_csv(r"diabetes.csv")

In [ ]:
dataset.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
dataset.isnull().mean() # проверка пропущенных значений

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
Insulin                     0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64

In [9]:
dataset.columns = [col.lower() for col in dataset.columns]
dataset.columns

Index(['pregnancies', 'glucose', 'bloodpressure', 'skinthickness', 'insulin',
       'bmi', 'diabetespedigreefunction', 'age', 'outcome'],
      dtype='object')

In [10]:
# Разделение X (data features) и y (outcome)

X = dataset.drop(columns=['outcome'])
y = dataset['outcome']

## EDA - исследовательский анализ данных 

(какие признаки, квантили/квартили, пустые ячейки, выбросы)

In [18]:
px.box(X)

In [11]:
X.describe(percentiles=[0.1, 0.25,0.5, 0.75, 0.9]).T

,count,mean,std,min,10%,25%,50%,75%,90%,max
pregnancies,768.0,3.845052,3.369578,0.000,0.000,1.00000,3.0000,6.00000,9.0000,17.00
glucose,768.0,120.894531,31.972618,0.000,85.000,99.00000,117.0000,140.25000,167.0000,199.00
bloodpressure,768.0,69.105469,19.355807,0.000,54.000,62.00000,72.0000,80.00000,88.0000,122.00
skinthickness,768.0,20.536458,15.952218,0.000,0.000,0.00000,23.0000,32.00000,40.0000,99.00
insulin,768.0,79.799479,115.244002,0.000,0.000,0.00000,30.5000,127.25000,210.0000,846.00
bmi,768.0,31.992578,7.884160,0.000,23.600,27.30000,32.0000,36.60000,41.5000,67.10
diabetespedigreefunction,768.0,0.471876,0.331329,0.078,0.165,0.24375,0.3725,0.62625,0.8786,2.42
age,768.0,33.240885,11.760232,21.000,22.000,24.00000,29.0000,41.00000,51.0000,81.00


## Подготовка данных

In [12]:
# нормализация
# MinMax
# StandardScalar
# Robast
scalar_test = StandardScaler()
features_arr = scalar_test.fit_transform(X, y)
print(features_arr)
normalised_X = pd.DataFrame(features_arr, columns=X.columns)
print(normalised_X)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
     pregnancies   glucose  bloodpressure  skinthickness   insulin       bmi  \
0       0.639947  0.848324       0.149641       0.907270 -0.692891  0.204013   
1      -0.844885 -1.123396      -0.160546       0.530902 -0.692891 -0.684422   
2       1.233880  1.943724      -0.263941      -1.288212 -0.692891 -1.103255   
3      -0.844885 -0.998208      -0.160546       0.154533  0.123302 -0.494043   
4      -1.141852  0.504055      -1.504687       0.907270  0.765836  1.409746   
..           ...       ...  

In [13]:
px.box(normalised_X)

In [17]:
TEST_SIZE_DEFAULT = 0.2
TEST_SIZE = 0.25
RANDOM_SEED = 1234 
NEED_SHUFFLE = True
# Разделение на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_DEFAULT, random_state=RANDOM_SEED, shuffle=NEED_SHUFFLE)
X_train25, X_test25, y_train25, y_test25 = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, shuffle=NEED_SHUFFLE)

## Работа с KNN

In [ ]:
# Примените алгоритм KNN (k-ближайших соседей) для данных

In [ ]:
# Рассчитайте модель kNN для k = 5, 10, 15, 20, 25.

In [ ]:
# Укажите объем тестового набора данных равным 25% от общего количества данных

In [ ]:
# Укажите score модели (метрики) и постройте визуализацию обученной классификации для указанных k

## Работа с множественной линейной регрессией

## Работа с Алгоритмом 3 (например, DecisionTreeClassifier)